In [1]:
import numpy as np
import pandas as pd

#import pandas_profiling
import ydata_profiling
from ydata_profiling import ProfileReport
 
from collections import Counter

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.lm import Vocabulary

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

In [3]:
#import nltk
#nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/danil-
[nltk_data]     pass123/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stop_words = set(stopwords.words('russian'))

# Read

In [5]:
df_train = pd.read_json("dataset/train.json")

In [6]:
df_train.head(3)

,id,text,label,extracted_part
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,{'text': ['Поставщик должен предоставить обесп...
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...


In [7]:
def extracted_p_process(df_extracted_part,column):
    res = []
    for items in list(df_extracted_part[column]):
        if len(items)==1:
            res.append(items[0])
        else:
            raise Error("Only one fragment expected")
    return pd.Series(res)

In [8]:
df_extracted_part = pd.DataFrame.from_records(list(df_train['extracted_part']))
for c in df_extracted_part.columns:
    df_extracted_part[c] = extracted_p_process(df_extracted_part,c)

df_extracted_part.head(3)

,text,answer_start,answer_end
0,Размер обеспечения исполнения контракта 6593.2...,1279,1343
1,Поставщик должен предоставить обеспечение испо...,1222,1318
2,Размер обеспечения исполнения контракта 10.00%,1297,1343


In [9]:
df = pd.concat([df_train,df_extracted_part],axis=1)
df = df.drop(['extracted_part'],axis=1)

df.columns = pd.Index(['id', 'text', 'label', 'text_output', 'answer_start', 'answer_end'])
df.head(5)

,id,text,label,text_output,answer_start,answer_end
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...,1279,1343
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...,1222,1318
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1297,1343
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1304,1350
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1302,1348
